In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time, csv

URL = "https://m.work24.go.kr/hr/a/a/1100/trnnCrsInf.do?dghtSe=A&traingMthCd=A&tracseTme=1&endDate=20261212&keyword1=&keyword2=&pageSize=10&orderBy=ASC&startDate_datepicker=2025-12-12&currentTab=1&topMenuYn=&pop=&tracseId=AIG20250000535559&pageRow=10&totamtSuptYn=A&keywordTrngNm=&crseTracseSeNum=&keywordType=1&gb=&keyword=AI&kDgtlYn=&area=00%7C%EC%A0%84%EA%B5%AD+%EC%A0%84%EC%B2%B4&orderKey=2&mberSe=&kdgLinkYn=&srchType=all_type&totTraingTime=A&crseTracseSe=A%7C%ED%9B%88%EB%A0%A8%EC%9C%A0%ED%98%95+%EC%A0%84%EC%B2%B4&tranRegister=&mberId=&i2=A&pageId=2&programMenuIdentification=EBG020000510010&endDate_datepicker=2026-12-12&monthGubun=&pageOrder=2ASC&pageIndex=1&bgrlInstYn=&startDate=20251212&crseTracseSeKDT=&ncs=&gvrnInstt=&selectNCSKeyword=&action=trnnCrsInfPost.do"

OUT_CSV = "고용24_제목_상세내용_URL.csv"
HEADLESS = True  

# 목록/상세 셀렉터 
LIST_SEL = "#tab-panel-01 > div > div.list_status_info > div > div.company_title > h3 > a"
ACTIVE_BTN_SEL = "#tab-panel-01 > div > div.section_bottom > div > button.active"

DETAIL_TITLE_SEL = "#section1 > div > div.emp_sumup_wrp.mt16 > div > div.left_area.br.js_s > div.tit_area > div > strong"
DETAIL_DESC_SEL_PRIMARY = "#tab-panel01_1 > div.box_table.notC0080 > table > tbody > tr:nth-child(1) > td > p"
DETAIL_DESC_SEL_FALLBACK = "#traCrseinfo > table > tbody > tr:nth-child(1) > td"

def make_driver():
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1280,1200")

    d = webdriver.Chrome(options=opts)
    try:
        d.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    except:
        pass
    return d

def safe_text(driver, selector):
    try:
        return driver.find_element(By.CSS_SELECTOR, selector).text.strip()
    except:
        return ""

def get_detail_desc(driver):
    """
    상세내용을 우선 셀렉터 → 대체 셀렉터 순서로 추출
    """
    for sel in (DETAIL_DESC_SEL_PRIMARY, DETAIL_DESC_SEL_FALLBACK):
        try:
            txt = driver.find_element(By.CSS_SELECTOR, sel).text.strip()
            if txt:
                return txt
        except:
            pass
    return ""

def wait_list_ready(wait, driver):
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, LIST_SEL)) > 0)

driver = make_driver()
wait = WebDriverWait(driver, 50)

results = []
seen = set()

visited_pages = set()
last_total = 0

try:
    driver.get(URL)
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
    time.sleep(1.5)

    page_loop = 1

    while True:
        print(f"\n📄 목록 {page_loop}페이지 수집 중...")

        # 1) 목록 로딩 대기
        try:
            wait_list_ready(wait, driver)
        except TimeoutException:
            print("목록 로딩 타임아웃 → 종료")
            break

        # 2) 무한루프 방지
        try:
            current_btn = driver.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL)
            active_text = current_btn.text.strip()
        except:
            active_text = ""
        page_id = f"{active_text}|{driver.current_url}"
        if page_id in visited_pages:
            print(f"이미 방문한 페이지({active_text})로 돌아옴 → 종료")
            break
        visited_pages.add(page_id)

        # 3) 현재 페이지 항목 수 고정 후 인덱스로 순회(stale 방지)
        items = driver.find_elements(By.CSS_SELECTOR, LIST_SEL)
        n = len(items)
        print(f" → 이 페이지 항목 수: {n}")

        for idx in range(n):
            items = driver.find_elements(By.CSS_SELECTOR, LIST_SEL)
            if idx >= len(items):
                continue

            el = items[idx]
            list_title = el.text.strip()
            if not list_title or list_title in seen:
                continue

            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            time.sleep(0.25)

            base_handle = driver.current_window_handle
            before_handles = set(driver.window_handles)

            # 클릭 (href가 void여도 onclick 이동)
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.9)

            # 새 탭/창이 열리면 전환, 아니면 같은 탭으로 이동한 것으로 간주
            after_handles = set(driver.window_handles)
            new_handles = list(after_handles - before_handles)

            opened_new_tab = False
            if new_handles:
                driver.switch_to.window(new_handles[0])
                opened_new_tab = True

            # 상세 페이지 로딩 대기
            try:
                wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
                # 제목이 늦게 뜰 수 있어 조금 더 기다림
                wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, DETAIL_TITLE_SEL)) > 0)
            except:
                pass

            detail_url = driver.current_url
            detail_title = safe_text(driver, DETAIL_TITLE_SEL) or list_title
            detail_desc = get_detail_desc(driver)

            seen.add(list_title)
            results.append({
                "title": detail_title,
                "detail": detail_desc,
                "url": detail_url
            })

            print(f"    ✔ {len(results)}건: {detail_title[:50]}")

            # 탭 닫고 목록 복귀
            if opened_new_tab:
                driver.close()
                driver.switch_to.window(base_handle)
            else:
                driver.back()
                try:
                    wait_list_ready(wait, driver)
                except:
                    pass

            time.sleep(0.6)

        # 4) 수집 증가가 없으면 종료(안전장치)
        if len(results) == last_total:
            print("다음 페이지로 넘어가도 새 데이터 없음 → 종료")
            break
        last_total = len(results)

        # 5) 다음 페이지 이동
        try:
            current_btn = driver.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL)
            next_btn = current_btn.find_element(By.XPATH, "following-sibling::button[1]")
        except:
            print("다음 페이지 버튼 없음 → 종료")
            break

        prev_active = current_btn.text.strip()
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", next_btn)
        time.sleep(0.3)
        driver.execute_script("arguments[0].click();", next_btn)

        # 페이지가 실제로 바뀌는지 확인
        try:
            wait.until(lambda d: d.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL).text.strip() != prev_active)
        except:
            print("페이지가 바뀌지 않음 → 종료")
            break

        time.sleep(1.0)
        page_loop += 1

    # 6) CSV 저장
    with open(OUT_CSV, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["course_title", "course_detail", "course_url"])
        for r in results:
            writer.writerow([r["title"], r["detail"], r["url"]])

    print(f"저장 완료")
    print("행 개수:", len(results))

finally:
    driver.quit()


📄 목록 1페이지 수집 중...
  → 이 페이지 항목 수: 10
    ✔ 1건: 업무 단축 끝판왕! 생성형 AI 마스터 클래스
모집중
    ✔ 2건: 골드러시의 리바이스, 엔비디아
모집중
    ✔ 3건: 세상을 바꾸는 기술_4차 산업혁명 트렌드 배우기
모집중
    ✔ 4건: AI레볼루션 A부터 Z까지_산업편
모집중
    ✔ 5건: 인공지능 데이터로 미래를 탐구하다
모집중
    ✔ 6건: 챗GPT가 바꾸는 비즈니스의 미래
모집중
    ✔ 7건: [GPT] AI가 여는 새로운 세상
모집중

📄 목록 2페이지 수집 중...
  → 이 페이지 항목 수: 10
    ✔ 8건: 인공지능시대 보건의료 빅데이터 활용
모집중
    ✔ 9건: [ChatGPT] 영업제안서 10분만에 생성하고 칼퇴하기
모집중
    ✔ 10건: 당장 시작해야 하는 ChatGPT와 AI 스마트워크
모집중
    ✔ 11건: 파이썬으로 만드는 업무매크로 활용법
모집중
    ✔ 12건: 쉽게 배워 바로 쓰는 사회복지 글쓰기
모집중
    ✔ 13건: 일잘러의 AI 비밀레시피: 인공지능 툴로 업무를 더 멋있게
모집중
    ✔ 14건: 인공지능활용능력향상을 위한 데이터 처리
모집중
    ✔ 15건: [AI 김현욱 아나운서] 10분이면 따라하는 직장인 ChatGPT 바이블
모집중
    ✔ 16건: AI시대, 전직원이 활용할 수 있는 빅데이터 브랜드 마케팅
모집중
    ✔ 17건: 쉽게 배우고 적용하는 디지털 융합 AI 서비스
모집중

📄 목록 3페이지 수집 중...
  → 이 페이지 항목 수: 10
    ✔ 18건: 인공지능활용능력향상을 위한 데이터 모델링
모집중
    ✔ 19건: 바이브 코딩과 AI 어시스턴트로 완성하는 웹 페이지 제작
모집중
    ✔ 20건: [디자인 첫걸음]광고&편집디자인(포토샵, 일러스트, 인디자인)입문과정
모집중
    ✔ 21건: [누구나하는]시각&편집출판 디자인(포토샵,일러스트) GTQ 자격증 취득
모집중
    ✔ 22건: 에너지 절감기술 전문

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time, csv

URL = "https://sll.seoul.go.kr/lms/jce/doListJceOnlineView.do?course_gubun=1&main_se=jce&mnid=202412335776&s_keyword=AI"

OUT_CSV = "서울런4050_제목_상세내용_URL.csv"
HEADLESS = True  

# 목록/상세 셀렉터
LIST_SEL = "#moreBtn"
ACTIVE_BTN_SEL = "#cardList > div > a"

DETAIL_TITLE_SEL = "#container > div.detail_top > div > div > div.pc > h2"
DETAIL_DESC_SEL_PRIMARY = "#section_01 > div:nth-child(2)"

def make_driver():
    opts = Options()
    if HEADLESS:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--window-size=1280,1200")

    d = webdriver.Chrome(options=opts)
    try:
        d.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    except:
        pass
    return d

def safe_text(driver, selector):
    try:
        return driver.find_element(By.CSS_SELECTOR, selector).text.strip()
    except:
        return ""

def get_detail_desc(driver):
    """
    상세내용을 우선 셀렉터 → 대체 셀렉터 순서로 추출
    """
    for sel in (DETAIL_DESC_SEL_PRIMARY):
        try:
            txt = driver.find_element(By.CSS_SELECTOR, sel).text.strip()
            if txt:
                return txt
        except:
            pass
    return ""

def wait_list_ready(wait, driver):
    wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, LIST_SEL)) > 0)

driver = make_driver()
wait = WebDriverWait(driver, 50)

results = []
seen = set()

visited_pages = set()
last_total = 0

try:
    driver.get(URL)
    wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
    time.sleep(1.5)

    page_loop = 1

    while True:
        print(f"\n📄 목록 {page_loop}페이지 수집 중...")

        # 1) 목록 로딩 대기
        try:
            wait_list_ready(wait, driver)
        except TimeoutException:
            print("목록 로딩 타임아웃 → 종료")
            break

        # 2) 무한루프 방지: active 페이지 반복 감지
        try:
            current_btn = driver.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL)
            active_text = current_btn.text.strip()
        except:
            active_text = ""
        page_id = f"{active_text}|{driver.current_url}"
        if page_id in visited_pages:
            print(f"이미 방문한 페이지({active_text})로 돌아옴 → 종료")
            break
        visited_pages.add(page_id)

        # 3) 현재 페이지 항목 수 고정 후 인덱스로 순회(stale 방지)
        items = driver.find_elements(By.CSS_SELECTOR, LIST_SEL)
        n = len(items)
        print(f" → 이 페이지 항목 수: {n}")

        for idx in range(n):
            items = driver.find_elements(By.CSS_SELECTOR, LIST_SEL)
            if idx >= len(items):
                continue

            el = items[idx]
            list_title = el.text.strip()
            if not list_title or list_title in seen:
                continue

            driver.execute_script("arguments[0].scrollIntoView({block:'center'});", el)
            time.sleep(0.25)

            base_handle = driver.current_window_handle
            before_handles = set(driver.window_handles)

            # 클릭 (href가 void여도 onclick 이동)
            driver.execute_script("arguments[0].click();", el)
            time.sleep(0.9)

            # 새 탭/창이 열리면 전환, 아니면 같은 탭으로 이동한 것으로 간주
            after_handles = set(driver.window_handles)
            new_handles = list(after_handles - before_handles)

            opened_new_tab = False
            if new_handles:
                driver.switch_to.window(new_handles[0])
                opened_new_tab = True

            # 상세 페이지 로딩 대기
            try:
                wait.until(lambda d: d.execute_script("return document.readyState") == "complete")
                # 제목이 늦게 뜰 수 있어 조금 더 기다림
                wait.until(lambda d: len(d.find_elements(By.CSS_SELECTOR, DETAIL_TITLE_SEL)) > 0)
            except:
                pass

            detail_url = driver.current_url
            detail_title = safe_text(driver, DETAIL_TITLE_SEL) or list_title
            detail_desc = get_detail_desc(driver)

            seen.add(list_title)
            results.append({
                "title": detail_title,
                "detail": detail_desc,
                "url": detail_url
            })

            print(f"    ✔ {len(results)}건: {detail_title[:50]}")

            # 탭 닫고 목록 복귀
            if opened_new_tab:
                driver.close()
                driver.switch_to.window(base_handle)
            else:
                driver.back()
                try:
                    wait_list_ready(wait, driver)
                except:
                    pass

            time.sleep(0.6)

        # 4) 수집이 안 늘면 종료
        if len(results) == last_total:
            print("다음 페이지로 넘어가도 새 데이터 없음 → 종료")
            break
        last_total = len(results)

        # 5) 다음 페이지 이동
        try:
            current_btn = driver.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL)
            next_btn = current_btn.find_element(By.XPATH, "following-sibling::button[1]")
        except:
            print("다음 페이지 버튼 없음 → 종료")
            break

        prev_active = current_btn.text.strip()
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", next_btn)
        time.sleep(0.3)
        driver.execute_script("arguments[0].click();", next_btn)

        # 페이지가 실제로 바뀌는지 확인
        try:
            wait.until(lambda d: d.find_element(By.CSS_SELECTOR, ACTIVE_BTN_SEL).text.strip() != prev_active)
        except:
            print("페이지가 바뀌지 않음 → 종료")
            break

        time.sleep(1.0)
        page_loop += 1

    # 6) CSV 저장
    with open(OUT_CSV, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["course_title", "course_detail", "course_url"])
        for r in results:
            writer.writerow([r["title"], r["detail"], r["url"]])

    print(f"CSV 저장 완료")
    print("행 개수:", len(results))

finally:
    driver.quit()


📄 목록 1페이지 수집 중...
  → 이 페이지 항목 수: 1
    ✔ 1건: 더보기 (
12
/
15
)
⛔ 다음 페이지 버튼 없음 → 종료

✅ CSV 저장 완료: 서울런4050_제목_상세내용_URL.csv
🎉 최종 수집 개수: 1
